# part.3 - TORCH.NN
---
> 신경망은 데이터에 대한 연산을 수행하는 계층(layer)/모듈(module)로 구성되어 있습니다.  
> torch.nn 네임스페이스는 신경망을 구성하는데 필요한 모든 구성 요소를 제공합니다.  
> PyTorch의 모든 모듈은 nn.Module 의 하위 클래스(subclass) 입니다.   
> 신경망은 다른 모듈(계층; layer)로 구성된 모듈입니다. 이러한 중첩된 구조는 복잡한 아키텍처를 쉽게 구축하고 관리할 수 있습니다.  


### 1. nn.Module
---
nn.Module는 모든 신경망 모듈의 기본 클래스이다.

모든 모델은 이 클래스의 하위 클래스로 지정해야 한다. 

In [1]:
# 사용 예시
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5)
        self.conv2 = nn.Conv2d(20, 20, 5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        return F.relu(self.conv2(x))

### 2. Dezero와의 연관성
---

사실 Dezero에서 Module을 찾아볼 순 없었다.

그래서 직접 코드를 뜯어본 결과

> 코드 : https://pytorch.org/docs/stable/generated/torch.nn.Module.html

가장 유사한 것이 Dezero의 Layer 클래스이다.


In [ ]:
class Layer:
    def __init__(self):
        self._params = set()

    def __setattr__(self, name, value): # object에 존재하는 속성의 값을 바꾸거나 새로운 속성을 생성할때마다 호출되는 특수 메서드
        if isinstance(value, (Parameter, Layer)):
            self._params.add(name)
        super().__setattr__(name, value)

    def __call__(self, *inputs): #함수 형태로 불리는 순간 forward 실행
        outputs = self.forward(*inputs)
        if not isinstance(outputs, tuple):
            outputs = (outputs, )
        self.inputs = [weakref.ref(x) for x in inputs]
        self.outputs = [weakref.ref(y) for y in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self,inputs):
        raise NotImplementedError()

    def params(self): # 파라미터를 반환함
        for name in self._params:
            obj = self.__dict__[name]

            if isinstance(obj, Layer):
                yield from obj.params()
            else:
                yield obj

    def cleargrads(self): # 파라미터 초기화
        for param in self.params():
            param.cleargrad()


class Linear(Layer): # Layer를 상속받아 사용하는 클래스 예시
    def __init__(self, out_size, nobias=False, dtype=np.float32, in_size=None):
        super().__init__() # 필수다 이거 안해주면 기초 클래스의 생성자를 가져오지 않는다는 건데 보통 기초 클래스에 속성을 정의하니까 그거 통쨰로 안쓴다고 하는 것과 다른게 없다
        self.in_size = in_size
        self.out_size = out_size
        self.dtype = dtype

        self.W = Parameter(None, name='W')
        if self.in_size is not None: # 만일 in_size가 지정되어 있지 않다면 나중으로 연기
            self._init_W()
        if nobias:
            self.b = None
        else:
            self.b = Parameter(np.zeros(out_size, dtype=dtype), name = 'b')

    def _init_W(self):
        I, O = self.in_size, self.out_size
        W_data = np.random(I,O).astype(self.dtype) * np.sprt(1/I)
        self.W.data = W_data

    def forward(self,x):
        if self.W.data is None:
            self.in_size = x.shape[1]
            self._init_W()

        y = F.linear(x, self.W, self.b)
        return y

왜 우리는 Layer 클래스를 만들었을까?

간단하게 말하자면 신경망은 매개변수(경사하강법 등의 최적화 기법에 의해 갱신되는 변수, '가중치'와 '편향'이 이에 해당)를 다뤄야 하는데 

신경망 층이 깊어지면 깊어질수록 매개변수가 어마어마하게 많아질거다

그런데 그 매개변수를 모두 코드에서 따로 관리해주자니 너무 복잡하다.

따라서 매개변수를 담는 구조를 따로 만들어서 매개변수 관리를 자동화 하는 과정을 위해 만든게 Layer 클래스이다.



### 3. pytorch의 module
---

module 클래스를 상속받아 정의할때 유도 클래스는 반드시 __init__과 forward()는 반드시 정의하도록 되어 있다. (당연한 것이 모듈을 받은 모델이 함수처럼 실행될때 forward()를 이용하니까....)

예를 들면 다음과 같이 이용한다

In [1]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

NameError: name 'nn' is not defined

근데 파이토치는 파이토치 나름의 복잡한 시스템을 가진다.
> Module에 관한 파이토치 글 : https://pytorch.org/docs/stable/generated/torch.nn.Module.html

어마어마한 수의 메서드를 같이 상속받지만 보통은 __init__과 forward()만 받는다고 생각하면 편하다.

참고로 반.드.시 super().__init__을 해줘야 하는데 간단하게 Dezero의 예시만 보더라도 self._params = set()을 가져오기 위함이었다.

파이토치는 보다 복잡한 시스템으로 모듈을 만들기 때문에 정상적인 모듈을 만들고 싶다면 그냥 기초클래스의 생성자를 죄다 가져와야 정상적인 모듈이 만들어진다.

참고로 생성자 안가져오면 'AttributeError: cannot assign module before Module.__init__() call' 이란 오류가 뜨면서 제발 가져오라고 한다.

